# Custom State for Memory Management

This notebook demonstrates how to create custom agent state schemas to track additional information beyond just conversation messages.

## Key Concepts
- **Default AgentState**: Only tracks messages[]
- **Custom AgentState**: Messages + user preferences + session data + task history
- **State Persistence**: Within and between conversations
- **Memory Management**: Context-aware responses based on history

## Benefits of Custom State
- **Persistent User Preferences**: Remember settings across conversations
- **Context-Aware Responses**: Better personalization
- **Session Management**: Handle complex workflows
- **Task Tracking**: Remember what user has done before

## Prerequisites

Make sure you have the required packages installed:

```bash
pip install langchain langchain-community langchain-core langgraph pydantic
ollama pull qwen3
ollama serve
```

In [ ]:
# Import required modules
from typing import TypedDict, Dict, Any, List, Optional
from typing_extensions import Annotated
from datetime import datetime, timedelta
from langchain_ollama import ChatOllama
from langchain.agents import create_agent, AgentState
from langgraph.graph.message import add_messages
import tools
import json

## Understanding Default AgentState

First, let's understand what the default AgentState provides:

In [ ]:
print("=== Default AgentState ===")

# Default AgentState only tracks messages
print("Default AgentState structure:")
print("  - messages: List of conversation messages")
print("  - That's it!")

print("\nLimitations of default state:")
print("  ✗ No user preferences memory")
print("  ✗ No session context")
print("  ✗ No task history")
print("  ✗ No personalization data")
print("  ✗ Conversation ends = everything forgotten")

# Create a basic agent with default state
model = ChatOllama(model="qwen3")
basic_agent = create_agent(model, tools=[tools.helper_tool])

print("\n✓ Basic agent created with default state")

## Creating Custom State Schema

Let's define a comprehensive custom state that tracks much more information:

In [ ]:
print("=== Custom State Schema Definition ===")

# Define custom state schema with rich information
class CustomAgentState(AgentState):
    """Extended agent state with comprehensive memory management."""
    
    # Required: conversation history (from base AgentState)
    messages: Annotated[list, add_messages]
    
    # User Profile and Preferences
    user_preferences: dict = {}
    user_profile: dict = {}
    
    # Session Management
    session_data: dict = {}
    session_id: Optional[str] = None
    session_start_time: Optional[datetime] = None
    
    # Task and Interaction History
    task_history: List[dict] = []
    interaction_count: int = 0
    
    # Context and Memory
    conversation_context: dict = {}
    long_term_memory: dict = {}
    
    # Analytics and Behavior
    user_behavior_patterns: dict = {}
    topics_discussed: List[str] = []
    
    # Workflow State
    current_workflow: Optional[str] = None
    workflow_step: int = 0
    workflow_data: dict = {}

print("✓ CustomAgentState defined with comprehensive tracking:")
print("\nUser Data:")
print("  - user_preferences: Settings and preferences")
print("  - user_profile: Basic user information")
print("\nSession Management:")
print("  - session_data: Current session information")
print("  - session_id: Unique session identifier")
print("  - session_start_time: When session began")
print("\nHistory and Memory:")
print("  - task_history: Previous tasks and outcomes")
print("  - conversation_context: Current conversation context")
print("  - long_term_memory: Persistent memory across sessions")
print("\nBehavior Analysis:")
print("  - user_behavior_patterns: Usage patterns")
print("  - topics_discussed: Subject matter tracking")
print("\nWorkflow Management:")
print("  - current_workflow: Active workflow name")
print("  - workflow_step: Current step in workflow")
print("  - workflow_data: Workflow-specific data")

## State Management Utilities

Let's create utilities to help manage our custom state:

In [ ]:
print("=== State Management Utilities ===")

class StateManager:
    """Utility class for managing custom agent state."""
    
    @staticmethod
    def initialize_session(user_id: str = None) -> dict:
        """Initialize a new session state."""
        session_id = f"session_{datetime.now().strftime('%Y%m%d_%H%M%S')}"
        
        return {
            "messages": [],
            "user_preferences": {},
            "user_profile": {"user_id": user_id} if user_id else {},
            "session_data": {
                "created_at": datetime.now(),
                "last_activity": datetime.now(),
                "interaction_count": 0
            },
            "session_id": session_id,
            "session_start_time": datetime.now(),
            "task_history": [],
            "interaction_count": 0,
            "conversation_context": {},
            "long_term_memory": {},
            "user_behavior_patterns": {},
            "topics_discussed": [],
            "current_workflow": None,
            "workflow_step": 0,
            "workflow_data": {}
        }
    
    @staticmethod
    def update_preferences(state: dict, preferences: dict) -> dict:
        """Update user preferences in state."""
        state["user_preferences"].update(preferences)
        return state
    
    @staticmethod
    def add_task_to_history(state: dict, task: dict) -> dict:
        """Add a completed task to history."""
        task_entry = {
            "timestamp": datetime.now(),
            "task": task,
            "interaction_id": state["interaction_count"]
        }
        state["task_history"].append(task_entry)
        return state
    
    @staticmethod
    def update_context(state: dict, context_key: str, context_value: Any) -> dict:
        """Update conversation context."""
        state["conversation_context"][context_key] = context_value
        return state
    
    @staticmethod
    def start_workflow(state: dict, workflow_name: str, initial_data: dict = None) -> dict:
        """Start a new workflow."""
        state["current_workflow"] = workflow_name
        state["workflow_step"] = 1
        state["workflow_data"] = initial_data or {}
        return state
    
    @staticmethod
    def advance_workflow(state: dict, step_data: dict = None) -> dict:
        """Advance to next workflow step."""
        if state["current_workflow"]:
            state["workflow_step"] += 1
            if step_data:
                state["workflow_data"].update(step_data)
        return state
    
    @staticmethod
    def get_session_summary(state: dict) -> dict:
        """Get a summary of the current session."""
        duration = datetime.now() - state.get("session_start_time", datetime.now())
        
        return {
            "session_id": state.get("session_id"),
            "duration_minutes": duration.total_seconds() / 60,
            "interactions": state.get("interaction_count", 0),
            "tasks_completed": len(state.get("task_history", [])),
            "topics_discussed": len(state.get("topics_discussed", [])),
            "current_workflow": state.get("current_workflow"),
            "workflow_progress": state.get("workflow_step", 0)
        }

print("✓ StateManager utility class created with methods:")
print("  - initialize_session(): Create new session state")
print("  - update_preferences(): Manage user preferences")
print("  - add_task_to_history(): Track completed tasks")
print("  - update_context(): Manage conversation context")
print("  - start_workflow(): Begin guided workflows")
print("  - advance_workflow(): Progress through workflows")
print("  - get_session_summary(): Session analytics")

## Creating Agent with Custom State

Now let's create an agent that uses our custom state schema:

In [ ]:
print("=== Creating Agent with Custom State ===")

# Create agent with custom state schema
enhanced_agent = create_agent(
    model,
    tools=[tools.save_user_preference, tools.get_user_preference, tools.helper_tool],
    state_schema=CustomAgentState  # This enables custom state tracking
)

print("✓ Enhanced agent created with custom state schema")
print("  The agent can now track comprehensive user information")
print("  State persists throughout the conversation")
print("  Rich context enables personalized responses")

# Initialize a sample session
sample_state = StateManager.initialize_session("demo_user_123")
print(f"\n✓ Sample session initialized: {sample_state['session_id']}")

## Testing Custom State Features

Let's test various aspects of our custom state management:

In [ ]:
print("=== Testing Custom State Features ===")

# Test 1: Basic conversation with state tracking
print("\n--- Test 1: Basic Conversation with Enhanced State ---")

# Prepare enhanced state with user preferences
enhanced_state = {
    "messages": "I prefer technical explanations and detailed examples",
    "user_preferences": {
        "response_style": "technical",
        "verbosity": "detailed",
        "examples": "code-heavy",
        "expertise_level": "advanced"
    },
    "user_profile": {
        "user_id": "demo_user_123",
        "role": "software_developer",
        "experience_years": 5
    },
    "session_data": {
        "session_id": "demo_session_001",
        "start_time": datetime.now().isoformat(),
        "environment": "development"
    },
    "conversation_context": {
        "current_topic": "agent_development",
        "discussion_depth": "intermediate"
    },
    "topics_discussed": ["langchain", "agents", "custom_state"]
}

print("Enhanced state prepared with:")
print(f"  - User preferences: {len(enhanced_state['user_preferences'])} settings")
print(f"  - User profile: {enhanced_state['user_profile']['role']}")
print(f"  - Session context: {enhanced_state['session_data']['session_id']}")
print(f"  - Topics discussed: {', '.join(enhanced_state['topics_discussed'])}")

try:
    result = enhanced_agent.invoke(enhanced_state)
    print(f"\n✓ Agent responded with enhanced context awareness")
    print(f"Response: {result['messages'][-1].content[:100]}...")
    
    # Check if preferences were maintained
    maintained_prefs = result.get('user_preferences', {})
    print(f"\n✓ Preferences maintained: {len(maintained_prefs)} items")
    for key, value in maintained_prefs.items():
        print(f"  - {key}: {value}")
        
except Exception as e:
    print(f"✗ Error testing enhanced state: {e}")

## Workflow Management Demo

Let's demonstrate how custom state can manage complex workflows:

In [ ]:
print("=== Workflow Management Demo ===")

# Define a multi-step workflow
class ProjectSetupWorkflow:
    """Multi-step project setup workflow."""
    
    STEPS = {
        1: "project_requirements",
        2: "technology_selection", 
        3: "architecture_design",
        4: "development_plan",
        5: "resource_allocation"
    }
    
    @staticmethod
    def get_step_prompt(step: int, workflow_data: dict) -> str:
        """Get prompt for current workflow step."""
        step_name = ProjectSetupWorkflow.STEPS.get(step, "unknown")
        
        prompts = {
            1: "Let's define your project requirements. What type of application are you building?",
            2: f"Based on your project '{workflow_data.get('project_type', 'application')}', let's select appropriate technologies.",
            3: f"Now let's design the architecture for your {workflow_data.get('tech_stack', 'application')}.",
            4: "Let's create a development plan with milestones and timelines.",
            5: "Finally, let's determine the resources and team structure needed."
        }
        
        return prompts.get(step, "Workflow step not defined.")
    
    @staticmethod
    def is_complete(step: int) -> bool:
        """Check if workflow is complete."""
        return step > len(ProjectSetupWorkflow.STEPS)

print("✓ ProjectSetupWorkflow defined with 5 steps:")
for step, name in ProjectSetupWorkflow.STEPS.items():
    print(f"  {step}. {name.replace('_', ' ').title()}")

# Initialize workflow state
workflow_state = StateManager.initialize_session("workflow_user")
workflow_state = StateManager.start_workflow(
    workflow_state, 
    "project_setup",
    {"started_at": datetime.now().isoformat()}
)

print(f"\n✓ Workflow initialized: {workflow_state['current_workflow']}")
print(f"  Current step: {workflow_state['workflow_step']}")
print(f"  Step name: {ProjectSetupWorkflow.STEPS[workflow_state['workflow_step']]}")

## Simulating Workflow Progression

In [ ]:
print("=== Simulating Workflow Progression ===")

# Simulate progressing through workflow steps
workflow_responses = [
    {"step": 1, "response": "I'm building a web application for e-commerce", "data": {"project_type": "e-commerce web app"}},
    {"step": 2, "response": "I'd like to use React and Node.js", "data": {"tech_stack": "React + Node.js"}},
    {"step": 3, "response": "I prefer microservices architecture", "data": {"architecture": "microservices"}},
    {"step": 4, "response": "I need this completed in 3 months", "data": {"timeline": "3 months"}},
    {"step": 5, "response": "I have a team of 4 developers", "data": {"team_size": 4}}
]

current_workflow_state = workflow_state.copy()

for step_data in workflow_responses:
    current_step = current_workflow_state["workflow_step"]
    
    print(f"\n--- Step {current_step}: {ProjectSetupWorkflow.STEPS[current_step].replace('_', ' ').title()} ---")
    
    # Get step prompt
    step_prompt = ProjectSetupWorkflow.get_step_prompt(current_step, current_workflow_state["workflow_data"])
    print(f"System: {step_prompt}")
    
    # Simulate user response
    print(f"User: {step_data['response']}")
    
    # Update workflow data
    current_workflow_state["workflow_data"].update(step_data["data"])
    
    # Add to task history
    task = {
        "workflow": "project_setup",
        "step": current_step,
        "step_name": ProjectSetupWorkflow.STEPS[current_step],
        "user_input": step_data['response'],
        "data_collected": step_data['data']
    }
    current_workflow_state = StateManager.add_task_to_history(current_workflow_state, task)
    
    # Advance workflow
    current_workflow_state = StateManager.advance_workflow(current_workflow_state, step_data["data"])
    
    print(f"✓ Data collected: {step_data['data']}")
    print(f"✓ Advanced to step {current_workflow_state['workflow_step']}")

# Workflow completion
if ProjectSetupWorkflow.is_complete(current_workflow_state["workflow_step"]):
    print(f"\n🎉 Workflow '{current_workflow_state['current_workflow']}' completed!")
    print("\nCollected project information:")
    for key, value in current_workflow_state["workflow_data"].items():
        if key != "started_at":
            print(f"  - {key.replace('_', ' ').title()}: {value}")
    
    # Clear workflow state
    current_workflow_state["current_workflow"] = None
    current_workflow_state["workflow_step"] = 0
    
    print("\n✓ Workflow state cleared for next workflow")

## Session Analytics and Memory

Let's explore how custom state enables rich analytics and memory:

In [ ]:
print("=== Session Analytics and Memory ===")

# Update the state with more realistic data
current_workflow_state["interaction_count"] = 15
current_workflow_state["topics_discussed"] = [
    "project_planning", "e-commerce", "react", "nodejs", 
    "microservices", "team_management", "timelines"
]
current_workflow_state["user_behavior_patterns"] = {
    "preferred_response_length": "detailed",
    "frequently_used_tools": ["helper_tool", "save_user_preference"],
    "session_duration_avg": 25.5,  # minutes
    "workflow_completion_rate": 0.8
}

# Add some long-term memory
current_workflow_state["long_term_memory"] = {
    "previous_projects": [
        {"name": "Blog Platform", "tech": "Django + React", "completed": True},
        {"name": "Task Manager", "tech": "Vue.js + Express", "completed": True}
    ],
    "preferred_technologies": ["React", "Node.js", "PostgreSQL"],
    "learning_goals": ["microservices", "kubernetes", "system_design"],
    "past_challenges": ["database_optimization", "scalability", "deployment"]
}

# Get comprehensive session summary
session_summary = StateManager.get_session_summary(current_workflow_state)

print("=== Comprehensive Session Analytics ===")
print(f"\nSession Overview:")
print(f"  - Session ID: {session_summary['session_id']}")
print(f"  - Duration: {session_summary['duration_minutes']:.1f} minutes")
print(f"  - Interactions: {session_summary['interactions']}")
print(f"  - Tasks completed: {session_summary['tasks_completed']}")
print(f"  - Topics discussed: {session_summary['topics_discussed']}")

print(f"\nUser Behavior Patterns:")
for pattern, value in current_workflow_state["user_behavior_patterns"].items():
    print(f"  - {pattern.replace('_', ' ').title()}: {value}")

print(f"\nLong-term Memory:")
memory = current_workflow_state["long_term_memory"]
print(f"  - Previous projects: {len(memory['previous_projects'])}")
for project in memory['previous_projects']:
    status = "✓" if project['completed'] else "○"
    print(f"    {status} {project['name']} ({project['tech']})")

print(f"  - Preferred technologies: {', '.join(memory['preferred_technologies'])}")
print(f"  - Learning goals: {', '.join(memory['learning_goals'])}")
print(f"  - Past challenges: {', '.join(memory['past_challenges'])}")

print(f"\nTask History Summary:")
print(f"  - Total tasks recorded: {len(current_workflow_state['task_history'])}")
if current_workflow_state['task_history']:
    latest_task = current_workflow_state['task_history'][-1]
    print(f"  - Latest task: {latest_task['task']['step_name']} at {latest_task['timestamp']}")
    print(f"  - Data collected: {latest_task['task']['data_collected']}")

## Context-Aware Agent Responses

Let's show how custom state enables intelligent, context-aware responses:

In [ ]:
print("=== Context-Aware Agent Responses ===")

class ContextAwareResponseGenerator:
    """Generate responses based on comprehensive state context."""
    
    @staticmethod
    def analyze_user_context(state: dict) -> dict:
        """Analyze user context from state."""
        
        # Analyze experience level
        experience_indicators = {
            "previous_projects": len(state.get("long_term_memory", {}).get("previous_projects", [])),
            "tech_familiarity": len(state.get("long_term_memory", {}).get("preferred_technologies", [])),
            "workflow_completion": state.get("user_behavior_patterns", {}).get("workflow_completion_rate", 0)
        }
        
        # Determine experience level
        if experience_indicators["previous_projects"] >= 3 and experience_indicators["workflow_completion"] > 0.7:
            experience_level = "expert"
        elif experience_indicators["previous_projects"] >= 1:
            experience_level = "intermediate"
        else:
            experience_level = "beginner"
        
        # Analyze current focus
        recent_topics = state.get("topics_discussed", [])[-3:]  # Last 3 topics
        current_workflow = state.get("current_workflow")
        
        return {
            "experience_level": experience_level,
            "experience_indicators": experience_indicators,
            "recent_focus": recent_topics,
            "active_workflow": current_workflow,
            "session_engagement": state.get("interaction_count", 0)
        }
    
    @staticmethod
    def generate_personalized_response(user_query: str, context: dict) -> str:
        """Generate a response tailored to user context."""
        
        # Base response
        response_parts = []
        
        # Personalized greeting based on experience
        if context["experience_level"] == "expert":
            response_parts.append("Based on your extensive project experience,")
        elif context["experience_level"] == "intermediate":
            response_parts.append("Considering your development background,")
        else:
            response_parts.append("As you're getting started,")
        
        # Reference recent context
        if context["recent_focus"]:
            topics = ", ".join(context["recent_focus"][-2:])
            response_parts.append(f"and building on our discussion of {topics},")
        
        # Workflow awareness
        if context["active_workflow"]:
            response_parts.append(f"within your current {context['active_workflow']} workflow,")
        
        # Main response
        response_parts.append(f"here's how I'd address '{user_query}':")
        
        # Add experience-appropriate detail level
        if context["experience_level"] == "expert":
            response_parts.append("I'll provide technical details and advanced considerations.")
        elif context["experience_level"] == "intermediate":
            response_parts.append("I'll include practical examples and best practices.")
        else:
            response_parts.append("I'll explain step-by-step with clear examples.")
        
        return " ".join(response_parts)

# Test context-aware responses
print("\n--- Testing Context-Aware Responses ---")

context_analyzer = ContextAwareResponseGenerator()
user_context = context_analyzer.analyze_user_context(current_workflow_state)

print("User Context Analysis:")
print(f"  - Experience level: {user_context['experience_level']}")
print(f"  - Previous projects: {user_context['experience_indicators']['previous_projects']}")
print(f"  - Workflow completion rate: {user_context['experience_indicators']['workflow_completion']}")
print(f"  - Recent topics: {', '.join(user_context['recent_focus'])}")
print(f"  - Session engagement: {user_context['session_engagement']} interactions")

# Test different queries
test_queries = [
    "How should I handle database connections?",
    "What's the best way to deploy my application?",
    "How do I implement user authentication?"
]

print("\n--- Personalized Responses ---")
for i, query in enumerate(test_queries, 1):
    response = context_analyzer.generate_personalized_response(query, user_context)
    print(f"\n{i}. Query: '{query}'")
    print(f"   Response: {response}")

## State Persistence Patterns

Let's explore how to persist state between sessions:

In [ ]:
print("=== State Persistence Patterns ===")

class StatePersistence:
    """Handle state persistence and restoration."""
    
    @staticmethod
    def serialize_state(state: dict) -> str:
        """Serialize state for storage."""
        # Convert datetime objects to strings for JSON serialization
        serializable_state = {}
        
        for key, value in state.items():
            if isinstance(value, datetime):
                serializable_state[key] = value.isoformat()
            elif isinstance(value, list) and value and isinstance(value[0], dict):
                # Handle task history with datetime objects
                serializable_list = []
                for item in value:
                    if isinstance(item, dict) and 'timestamp' in item:
                        item_copy = item.copy()
                        if isinstance(item_copy['timestamp'], datetime):
                            item_copy['timestamp'] = item_copy['timestamp'].isoformat()
                        serializable_list.append(item_copy)
                    else:
                        serializable_list.append(item)
                serializable_state[key] = serializable_list
            else:
                serializable_state[key] = value
        
        return json.dumps(serializable_state, indent=2)
    
    @staticmethod
    def deserialize_state(serialized_state: str) -> dict:
        """Deserialize state from storage."""
        state = json.loads(serialized_state)
        
        # Convert datetime strings back to datetime objects
        datetime_fields = ['session_start_time']
        
        for field in datetime_fields:
            if field in state and state[field]:
                state[field] = datetime.fromisoformat(state[field])
        
        # Handle task history timestamps
        if 'task_history' in state:
            for task in state['task_history']:
                if 'timestamp' in task and isinstance(task['timestamp'], str):
                    task['timestamp'] = datetime.fromisoformat(task['timestamp'])
        
        return state
    
    @staticmethod
    def extract_persistent_data(state: dict) -> dict:
        """Extract data that should persist between sessions."""
        persistent_fields = [
            'user_preferences',
            'user_profile', 
            'long_term_memory',
            'user_behavior_patterns',
            'task_history'  # Could be filtered by date
        ]
        
        persistent_data = {}
        for field in persistent_fields:
            if field in state:
                persistent_data[field] = state[field]
        
        return persistent_data
    
    @staticmethod
    def restore_to_new_session(persistent_data: dict, new_session_id: str) -> dict:
        """Create new session with restored persistent data."""
        new_state = StateManager.initialize_session()
        new_state['session_id'] = new_session_id
        
        # Restore persistent data
        for key, value in persistent_data.items():
            new_state[key] = value
        
        return new_state

# Demonstrate state persistence
print("\n--- State Persistence Demo ---")

persistence_manager = StatePersistence()

# 1. Serialize current state
print("1. Serializing current session state...")
serialized = persistence_manager.serialize_state(current_workflow_state)
print(f"   ✓ State serialized ({len(serialized)} characters)")

# 2. Extract persistent data
print("\n2. Extracting persistent data...")
persistent_data = persistence_manager.extract_persistent_data(current_workflow_state)
print(f"   ✓ Persistent data extracted:")
for key in persistent_data.keys():
    if isinstance(persistent_data[key], list):
        print(f"     - {key}: {len(persistent_data[key])} items")
    elif isinstance(persistent_data[key], dict):
        print(f"     - {key}: {len(persistent_data[key])} fields")
    else:
        print(f"     - {key}: {type(persistent_data[key]).__name__}")

# 3. Simulate new session with restored data
print("\n3. Creating new session with restored data...")
new_session_state = persistence_manager.restore_to_new_session(persistent_data, "session_002")
print(f"   ✓ New session created: {new_session_state['session_id']}")
print(f"   ✓ Preferences restored: {len(new_session_state.get('user_preferences', {}))} items")
print(f"   ✓ Long-term memory restored: {len(new_session_state.get('long_term_memory', {}))} categories")
print(f"   ✓ Task history restored: {len(new_session_state.get('task_history', []))} tasks")

# 4. Compare session summaries
print("\n4. Comparing session summaries...")
old_summary = StateManager.get_session_summary(current_workflow_state)
new_summary = StateManager.get_session_summary(new_session_state)

print("   Original session:")
print(f"     - Tasks completed: {old_summary['tasks_completed']}")
print(f"     - Topics discussed: {old_summary['topics_discussed']}")

print("   New session (with restored data):")
print(f"     - Tasks completed: {new_summary['tasks_completed']} (restored)")
print(f"     - Fresh interaction count: {new_summary['interactions']}")
print(f"     - New session ID: {new_summary['session_id']}")

print("\n✓ State persistence successfully demonstrated!")
print("  User data and history preserved across sessions")
print("  Session-specific data reset for new interaction")

## Best Practices Summary

### State Design Principles

1. **Separation of Concerns**
   - **Session Data**: Temporary, session-specific information
   - **User Preferences**: Settings that persist between sessions
   - **Long-term Memory**: Historical data and learned patterns
   - **Workflow State**: Current process and step tracking

2. **Data Organization**
   - Group related data in dictionaries
   - Use consistent naming conventions
   - Include metadata (timestamps, IDs, etc.)
   - Plan for data evolution and migration

3. **Performance Considerations**
   - Limit state size for active sessions
   - Archive old task history periodically
   - Use efficient serialization formats
   - Cache frequently accessed data

### Memory Management Strategies

1. **Short-term Memory** (Session)
   - Current conversation context
   - Active workflow state
   - Temporary variables and flags

2. **Medium-term Memory** (User Session)
   - User preferences and settings
   - Recent interaction patterns
   - Current project context

3. **Long-term Memory** (Persistent)
   - User profile and history
   - Learned behavior patterns
   - Completed workflows and outcomes

### Implementation Guidelines

1. **State Schema Design**
   - Start with essential fields
   - Add complexity gradually
   - Document field purposes
   - Plan for backward compatibility

2. **Persistence Strategy**
   - Identify what needs to persist
   - Choose appropriate storage (files, database, etc.)
   - Handle serialization/deserialization properly
   - Implement data migration for schema changes

3. **Privacy and Security**
   - Sanitize sensitive data before storage
   - Encrypt persistent user data
   - Implement data retention policies
   - Provide user control over their data

## Use Cases for Custom State

### Educational Tutors
- Track learning progress and mastery
- Adapt difficulty based on performance
- Remember preferred learning styles
- Maintain curriculum progress

### Customer Service Agents
- Remember customer history and preferences
- Track ongoing support tickets
- Maintain context across conversations
- Escalation patterns and resolution history

### Personal Assistants
- User's schedule and commitments
- Personal preferences and habits
- Project and goal tracking
- Contact and relationship management

### Development Assistants
- Project history and tech stack preferences
- Code patterns and architecture choices
- Learning goals and skill development
- Workflow and methodology preferences

## Conclusion

Custom state management transforms agents from simple question-answering tools into sophisticated, memory-enabled assistants that:

- **Remember**: User preferences, history, and context
- **Learn**: Patterns and behaviors over time
- **Adapt**: Responses based on accumulated knowledge
- **Guide**: Users through complex, multi-step workflows
- **Persist**: Important information across sessions

This creates a foundation for building truly intelligent, personalized agent experiences that improve over time.